# Análisis de Arquitectura: README vs Implementación ML

Este documento presenta un análisis de la carpeta `business_backend/ml` en relación con lo especificado en el `README.md`, enfocado en el requerimiento de un **Asistente de Reconocimiento de Productos (Multiagente ML + Planners)**.

## 1. Análisis de Estructura: Carpeta `ml` vs `README.md`

La estructura física de archivos coincide con la documentación, pero existen diferencias importantes en el nivel de implementación.

| Componente | Estado en carpeta `ml/` | Alineación con README |
| :--- | :--- | :--- |
| **Estructura General** | **Correcto**. Contiene `models`, `preprocessing`, `serving`, `training`. | **Alineado**. Coincide con la sección "Machine Learning Module" del README. |
| **Reconocimiento de Productos** | **Esqueleto**. Existe `models/image_classifier.py` pero los métodos (`load`, `predict`) están vacíos (`pass`). | **Técnicamente Alineado**. El archivo existe donde se promete, pero la funcionalidad lógica aún no está implementada. |
| **Multi-agente / Planners** | **No Presente en ML**. No hay código de agentes dentro de `ml/`. | **Alineado**. El README ubica correctamente a los agentes y herramientas en la carpeta `llm/` para mantener la separación de responsabilidades. |

### Diagrama de la Arquitectura Actual

```mermaid
graph TD
    Examples[Usuario] --> GraphQL
    GraphQL --> SearchService[Search Service (Planner)]
    
    subgraph "Módulo LLM (Agentes)"
    SearchService --> LLM[LLM Provider]
    LLM --> Tools[Herramientas / Tools]
    Tools --> ProductSearch[ProductSearchTool]
    Tools -.->|Faltante| ImageTool[ImageRecognitionTool]
    end
    
    subgraph "Módulo ML (Lógica Pura)"
    ImageTool -.-> InferenceService
    InferenceService --> Registry
    Registry --> ImageModel[ImageClassifier]
    end
```

## 2. Discrepancia: ¿Dónde está la parte Multiagente + Planners?

El requerimiento menciona un "Asistente de reconocimiento de productos (multiagente ML + planners)". Analizando el código:

### La parte ML (Reconocimiento)
- **Ubicación**: `ml/models/image_classifier.py`
- **Estado Actual**: Es un cascarón vacío. 
- **Lo que falta**: Implementar la carga del modelo (TensorFlow/PyTorch) y la lógica de predicción real.

### La parte Agente (Planner)
- **Ubicación**: NO debe estar en `ml`. Vive correctamente en `services/search_service.py` y `llm/tools`.
- **Estado Actual**: El `SearchService` actúa como el orquestador (Planner) usando LangChain.
- **El Eslabón Perdido**: Actualmente el Agente solo tiene una herramienta (`ProductSearchTool` para texto). No tiene forma de "ver" imágenes porque no existe aún una `ImageRecognitionTool` conectada al módulo ML.

## 3. Conclusiones y Pasos a Seguir

Para completar la tarea del asistente de reconocimiento visual, se deben realizar los siguientes pasos:

1.  **Implementar el Modelo ML**: Rellenar el código en `ml/models/image_classifier.py` para que cargue los pesos entrenados y procese imágenes.
2.  **Crear la Herramienta de Interfaz**: Crear una nueva herramienta en `llm/tools/` (ej. `image_recognition_tool.py`) que sirva de puente llamando al `InferenceService`.
3.  **Actualizar el Planner**: Modificar `services/search_service.py` para registrar esta nueva herramienta en el LLM, permitiéndole decidir cuándo usar reconocimiento visual.